In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import wfdb
import glob
import os

%run ./utils.ipynb

https://www.kaggle.com/code/l33tc0d3r/ecg-heartbeat-categorization-using-cnn

In [ ]:

'''We exclude specific annotations/beats; for more details about annotations go to: 
https://archive.physionet.org/physiobank/annotations.shtml#aux''' 
def exclude(labels,leadML2):
    exclude_char = ['~','|','!','[',']','"','no','+','x']
    zero_list=[]
    for i in exclude_char:
        for j in range(len(labels)):
            if labels[j]==i:
                labels[j] = 0  # Finds char from 'exclude_char' and replaces them with zero
                zero_list.append(j)  #find the indexes where labels=0 
    leadML2 = [leadML2[i] for i in range(0, len(labels)) if i not in zero_list] 
    labels = [labels[i] for i in range(0, len(labels)) if i not in zero_list] 
    
    
    new_labels = []; signals=[]
    for j in range(len(labels)):
        if labels[j]=='N':
            label = 0
            new_labels.append(label)  # N
            signals.append(leadML2[j])
        elif labels[j]=='A':
            label = 1
            new_labels.append(label)  #'SVEB'
            signals.append(leadML2[j])
        elif labels[j]=='L':
            label = 2
            new_labels.append(label) # 'VEB'
            signals.append(leadML2[j])

        elif labels[j]=='R':
            label = 3
            new_labels.append(label) #'F'
            signals.append(leadML2[j])
    return new_labels,signals

In [ ]:
'''Download the database from: https://physionet.org/content/mitdb/1.0.0/'''
paths = glob.glob(r'C:\Users\1938759.IKB\OneDrive - Brunel University London\05 Datasets\mit-bih-arrhythmia-database-1.0.0\mit-bih-arrhythmia-database-1.0.0\*.atr')             
paths.pop(1)

[leadML2, _, labels] = segmentation(paths)

# Normalise signals between [-1 and 1]
leadML2 = normalise(leadML2, -1, 1)

# Filter the signals
leadML2 = Savitzky_Golay(leadML2, fs=360)

# Exclude the signals that have the annotations in exclude_char 
[labels,signals] = exclude(labels,leadML2)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(np.array(signals), np.array(labels), test_size = 0.3, shuffle=True)
print(f'The shape of the training is: {np.shape(np.array(X_train))}')
print(f'The shape of the test is: {np.shape(np.array(X_test))}')

In [ ]:
X_train = np.array(X_train).reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = np.array(X_test).reshape(X_test.shape[0], X_test.shape[1], 1)

print(f'The shape of the training is: {np.shape(np.array(X_train))}')
print(f'The shape of the test is: {np.shape(np.array(X_test))}')

In [ ]:
y_train

In [ ]:
np.shape(X_train)

In [ ]:
input_shape = (X_train.shape[2],1)
input_shape

In [ ]:
unique, counts = np.unique(y_train, return_counts=True)
fig =plt.figure(figsize=[3,2])
plt.bar(unique,counts)
plt.title("Class distribution- training set")
plt.xlabel('Class')
plt.ylabel('Distribution')

In [ ]:
from tensorflow.keras import Sequential,utils
from tensorflow.keras.layers import Flatten, Dense, Conv1D, MaxPool1D, Dropout

model = Sequential()

model.add(Conv1D(filters=64, kernel_size=(3,), padding='same', activation='relu', input_shape = (X_train.shape[1],1)))
#model.add(Conv1D(filters=32, kernel_size=(3,), padding='same', activation='relu')) 
#clf.add(Conv1D(filters=128, kernel_size=(5,), padding='same', activation='relu'))    

#model.add(MaxPool1D(pool_size=(3,), strides=2, padding='same'))
#model.add(Dropout(0.5))
model.add(Flatten())

model.add(Dense(units = 50, activation='relu'))
#clf.add(Dense(units = 1024, activation='relu'))

model.add(Dense(units = 4, activation='softmax'))

In [ ]:
from tensorflow.keras import Sequential,utils
from tensorflow.keras.layers import Flatten, Dense, Conv1D, MaxPool1D, Dropout, LSTM
model = Sequential()

model.add(LSTM(256,input_shape = (X_train.shape[1],1), return_sequences=True))
model.add(LSTM(128, return_sequences=True))
model.add(LSTM(64, return_sequences=False))

model.add(Dense(units = 4, activation='softmax'))

In [ ]:
model.compile(optimizer='adam', loss = 'sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(X_train, y_train, epochs = 10)

In [ ]:
# Prediction
y_pred = model.predict(X_test)

In [ ]:
acc = history.history['accuracy']
epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, color='red', label='Training acc')
plt.title('Training Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
y_lbl = [np.where(i == np.max(i))[0][0] for i in y_pred]
CM = confusion_matrix(y_test, y_lbl)

In [ ]:
ar_unique, i = np.unique(y_test, return_counts=True)
fig =plt.figure(figsize=[3,2])
plt.bar(unique,counts)
plt.title("Class distribution- training set")
plt.xlabel('Class')
plt.ylabel('Distribution')

In [ ]:
from sklearn.metrics import confusion_matrix,precision_score,recall_score,classification_report
print('classification_report :\n',classification_report(y_test,y_lbl))


In [ ]:
Precision_N = CM[0,0]/(CM[0,0]+(CM[0,1]+CM[0,2]+CM[0,3]))
Recall_N = CM[0,0]/(CM[0,0]+(CM[1,0]+CM[2,0]+CM[3,0]))
print(f'Precision and Recall for SR class {Precision_N}, {Recall_N}')

Precision_A = CM[1,1]/(CM[1,1]+(CM[1,2]+CM[1,3]+CM[1,0]))
Recall_A = CM[1,1]/(CM[1,1]+(CM[2,1]+CM[3,1]+CM[0,1]))
print(f'Precision and Recall for A class {Precision_A}, {Recall_A}')

Precision_L = CM[2,2]/(CM[2,2]+(CM[2,0]+CM[2,1]+CM[2,3]))
Recall_L = CM[2,2]/(CM[2,2]+(CM[0,2]+CM[1,2]+CM[3,2]))
print(f'Precision and Recall for LBBB class {Precision_L}, {Recall_L}')

Precision_R = CM[3,3]/(CM[3,3]+(CM[3,1]+CM[3,2]+CM[3,0]))
Recall_R = CM[3,3]/(CM[3,3]+(CM[1,3]+CM[2,3]+CM[0,3]))
print(f'Precision and Recall for RBBB class {Precision_R}, {Recall_R}')

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
y_lbl = [np.where(i == np.max(i))[0][0] for i in y_pred]
mat = confusion_matrix(y_test, y_lbl)
fig, ax = plt.subplots(figsize=(7,7))
sns.heatmap(mat, annot = True)

In [ ]:
# Measure the Accuracy Score

from sklearn import metrics

print("Accuracy score of the predictions: {0}".format(metrics.accuracy_score(y_lbl, y_test)))